In [34]:
import os, sys
sys.path = list(set([
    "../../lib/",
] + sys.path))
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Iterable, Any
from plotly import graph_objects as go, subplots as sp
from concurrent.futures import ProcessPoolExecutor as Exe
import networkx as nx
# import numba

from local.caching import load, save, save_exists
from local.figures import layout, xaxis_desc, yaxis_desc
# from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable

In [35]:
def _circulant(n, k):
    return nx.Graph(nx.circulant_graph(n, [i+3 for i in range(k)]))

def _scale_free(n, k, seed:int=0, g=None):
    return nx.Graph(nx.barabasi_albert_graph(n, k, seed=seed, initial_graph=g))

In [44]:
n = 25
SF = _scale_free(n, 4, seed=2)
MD = _circulant(n, 4)
# G = _scale_free(n, k)
    
pos_sf = nx.kamada_kawai_layout(SF)
pos_md = nx.kamada_kawai_layout(MD)

In [51]:
no_col = 'rgba(0, 0, 0, 0)'
axis_desc: dict = dict(
    linecolor=no_col, gridcolor=no_col, zerolinecolor=no_col, zerolinewidth=0,
    visible=False, range=(-1.1, 1.1), autorange=False,
)

def _plot(G: nx.Graph, pos: dict, c):
    n_plot = go.Scatter(
            x=[x for i, (x, y) in enumerate(pos.values())],
            y=[y for i, (x, y) in enumerate(pos.values())],
            mode="markers", showlegend=False,
            marker=dict(
                color=c,
                size=15,
            )
        )

    edges = []
    for a, b in G.edges:
        edges += [(pos[a]), (pos[b]), (None, None)]
    e_plot = go.Scatter(
        x=[x for x, y in edges], y=[y for x, y in edges],
        mode="lines", showlegend=False,
        line=dict(width=1, color="rgba(0, 0, 0, 0.2)"),
    )
    plots = (e_plot, n_plot)
    return plots

fig = sp.make_subplots(
    rows=1, cols=2, shared_xaxes=True, shared_yaxes=True, horizontal_spacing=0.05, vertical_spacing=0.02,
)

a, b = _plot(SF, pos_sf, "#F76E21")
fig.add_trace(a, row=1, col=1)
fig.add_trace(b, row=1, col=1)
a, b = _plot(MD, pos_md, "#32CD32")
fig.add_trace(a, row=1, col=2)
fig.add_trace(b, row=1, col=2)

_layout: Any = layout.copy()
_layout.update(dict(
    width=1300, height=700,
    xaxis=axis_desc, yaxis=axis_desc,
    xaxis2=axis_desc, yaxis2=axis_desc,
))
fig.update_layout(go.Layout(_layout))
fig.show()